In [3]:
import os
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import gzip
import pickle as pkl

def loadall_results2(path, n_folds):
    regret = []
    history_actions = []
    history_outcomes = []
    perf = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds+1):
            try:
                data = pkl.load(f)
            except EOFError:
                break

            if type(data) == dict:
                regret.append( data['regret'] ) 
                history_actions.append( data['action_history'] )
                history_outcomes.append( data['outcome_history'] ) 
                perf.append( data['pred'] ) 

    return regret, history_actions, history_outcomes, perf

In [25]:
n_folds = 25
horizon = 9999
context = 'MNISTbinary'
model = 'MLP'
case = 'case1'
agent_name = 'EEneuralcbpside_v6' #ineural6, neuronal6

direct = './results/'
path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case,model,context,horizon,n_folds,agent_name) )
regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)


In [44]:
# Process the list to create a format suitable for DataFrame


# Create DataFrame
df = pd.DataFrame(processed_data)


In [4]:
def format_perf(perf, model, dataset, l_label):
    processed_data = []
    i = 0
    for item in perf:
        processed_item = {}
        for key in item:
            processed_item[f'accuracy_{key}'] = item[key]['accuracy']
            processed_item[f'f1_{key}'] = item[key]['f1']
        processed_item['Seed'] = i
        processed_item['Model'] = model
        processed_item['Dataset'] = dataset
        processed_item['Approach'] = l_label
        i = i+1
        processed_data.append(processed_item)
    return processed_data



n_folds = 26
horizon = 9999
model = 'MLP'

material = {
    #'EEneuralcbpside_v5': {'color': [255, 255, 0], 'label': 'EEneuralcbpside_v5'},  # Red
    'EEneuralcbpside_v6': {'color': [255, 0, 0], 'label': 'Neural-CBP'},  # Red
    'ineural6': {'color': [51, 255, 255], 'label':'IneurAL (official)'},                    # Yellow
    'ineural3': {'color': [0, 0, 255], 'label':'IneurAL (tuned)'},                    # Cyan
    'neuronal6': {'color': [255, 0, 255], 'label':'Neuronal (official)'},                  # Magenta
    'neuronal3': {'color': [160, 160, 160], 'label':'Neuronal (tuned)'},                   # Orange
    'margin': {'color': [160, 160, 160], 'label':'Margin'},
    'cesa': {'color': [0, 0, 255], 'label':'Cesa'},
}

fig = go.Figure( )

data_models = {}
perfs_final = {}
for model in ['MLP']:  # 'LeNet'

    if model == 'MLP':
        n_folds = 25
        datasets = ['MNISTbinary', 'MagicTelescope', 'adult', ] # 'MNIST', 'FASHION', 'covertype', 'shuttle',
    # else:
    #     datasets = ['MNIST', 'FASHION', 'CIFAR10',]
    #     n_folds = 25

    data_regrets = {} 
    data_perfs = {} 
    for data in datasets: 

        # if data in ['MNIST', 'FASHION', 'CIFAR10']:
        #     case = 'case2' 
        # elif data in ['covertype', 'shuttle']:
        #     case = 'game_case_seven'
        # else:
        case = 'case1'
        
        final_regrets = {}
        final_perfs = {}
        for agent_name in material.keys():

            color, l_label = material[agent_name]['color'], material[agent_name]['label']

            r,g,b = color

            # try:
            direct = './results/'
            path = os.path.join(direct, '{}_{}_{}_{}_{}_{}.pkl.gz'.format(case, model, data, horizon,n_folds,agent_name) )
            print(path)
            regret, action_history,outcome_history, perf = loadall_results2(path, n_folds)
            regret = np.array(regret)
            regret = regret#[1:]
            # regret = regret.astype(np.float32)
            # print(regret.shape)
            
            final_regrets[l_label] = regret[:,-1] 
            final_perfs[l_label] = pd.DataFrame( format_perf(perf, model, data, l_label) )
            # except:
            #     print('hey')
            #     regret = np.zeros((n_folds,horizon))

        if data == 'MNISTbinary':
            data = 'MNISTbinary'
        if data == 'FASHION':
            data = 'Fashion'
            
        data_regrets[data] = final_regrets

        dataframes_list = list(final_perfs.values())
        concatenated_df = pd.concat(dataframes_list, axis=0)
        data_perfs[data] = concatenated_df
        
    data_models[model] = data_regrets
    dataframes_list = list(data_perfs.values())
    concatenated_df = pd.concat(dataframes_list, axis=0)

# fig.show()
# fig.write_image("./figures/case1_{}_{}.pdf".format(model, context) )

./results/case1_MLP_MNISTbinary_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_ineural6.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_ineural3.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_neuronal6.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_neuronal3.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_margin.pkl.gz
./results/case1_MLP_MNISTbinary_9999_25_cesa.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_ineural6.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_ineural3.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_neuronal6.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_neuronal3.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_margin.pkl.gz
./results/case1_MLP_MagicTelescope_9999_25_cesa.pkl.gz
./results/case1_MLP_adult_9999_25_EEneuralcbpside_v6.pkl.gz
./results/case1_MLP_adult_9999_25_ineural6.pkl.gz
./results/case1_MLP_adult_9999_25_ineural3.pkl.gz
./results/case1_MLP_adu

In [5]:
import pandas as pd
# pd.DataFrame(data_regrets)
# data_regrets

samples = []
# Iterate over the dictionary to extract data
for model, dataset in data_models.items():
    for data, approach in dataset.items():
        for appr, values in approach.items():
            i = 0
            for val in values:
                # print(i, appr,val)
                samples.append([i, model, data, appr, val])
                i = i+1

# Create a DataFrame
df = pd.DataFrame(samples, columns=['Seed','Model', 'Dataset', 'Approach', 'Value'])
df = pd.merge(df, concatenated_df, on=['Model', 'Dataset', 'Approach', 'Seed'])

# Calculate mean and standard error for each group
# grouped = df.groupby(['Model', 'Dataset', 'Approach'])
# mean = grouped['Value'].mean().reset_index()
# std_error = grouped['Value'].std() / np.sqrt(grouped['Value'].count())
# std_error = std_error.reset_index()
# # Merge the mean and standard error into a single DataFrame
# result = pd.merge(mean, std_error, on=['Model', 'Dataset', 'Approach'])
# result.rename(columns={'Value_x': 'Mean', 'Value_y': 'StdError'}, inplace=True)
# result['merge'] = result['Model'] + result['Approach']
# result['merge2'] = result['Model'] + result['Dataset']

In [107]:
df.to_csv('experiment.csv', index=False)

In [6]:
import pandas as pd

# Load the dataframe from the provided file
# df = pd.read_csv('/mnt/data/experiment.csv')

# Columns to exclude from the aggregation
exclude_columns = ['Seed', 'Model', 'Dataset', 'Approach']

# Selecting the columns for which to calculate mean and std
agg_columns = [col for col in df.columns if col not in exclude_columns]

# Grouping by 'Model', 'Dataset', 'Approach' and calculating mean and std
grouped_df = df.groupby(['Model', 'Dataset', 'Approach'])[agg_columns].agg(['mean']) #, 'std'

grouped_df#.head()  # Display the first few rows of the result


Value accuracy_10     f1_10  \
                                             mean        mean      mean   
Model Dataset        Approach                                             
MLP   MNISTbinary    Cesa                 4857.56         NaN       NaN   
                     IneurAL (official)   3929.48    0.658688  0.642092   
                     IneurAL (tuned)      1726.96    0.643038  0.615288   
                     Margin               3216.72    0.492600  0.325144   
                     Neural-CBP           1351.92    0.611452  0.563043   
                     Neuronal (official)  1721.88    0.567744  0.516245   
                     Neuronal (tuned)     1721.44    0.590775  0.540633   
      MagicTelescope Cesa                 3578.52    0.449452  0.290726   
                     IneurAL (official)   9058.72    0.619334  0.591787   
                     IneurAL (tuned)      6225.28    0.630701  0.568581   
                     Margin               3459.84    0.501147  0.347732   
                     Neural-CBP           3469.84    0.615352  0.595573   
                     Neuronal (official)  4433.44    0.627134  0.537558   
                     Neuronal (tuned)     4439.04    0.611520  0.571841   
      adult          Cesa                 2695.56    0.473823  0.346161   
                     IneurAL (official)   9054.12    0.745679  0.655577   
                     IneurAL (tuned)      3972.84    0.655228  0.548743   
                     Margin               2845.24    0.632677  0.519022   
                     Neural-CBP           3445.20    0.592175  0.484292   
                     Neuronal (official)  4562.84    0.739866  0.635286   
                     Neuronal (tuned)     3668.36    0.767572  0.666641   

                                         accuracy_50     f1_50 accuracy_100  \
                                                mean      mean         mean   
Model Dataset        Approach                                                 
MLP   MNISTbinary    Cesa                        NaN       NaN          NaN   
                     IneurAL (official)     0.759192  0.754211     0.819428   
                     IneurAL (tuned)        0.770204  0.767658     0.828729   
                     Margin                 0.492600  0.325144     0.492600   
                     Neural-CBP             0.722312  0.712713     0.775408   
                     Neuronal (official)    0.753144  0.748138     0.778388   
                     Neuronal (tuned)       0.720904  0.711540     0.767248   
      MagicTelescope Cesa                   0.449452  0.290726     0.449452   
                     IneurAL (official)     0.662909  0.637522     0.687712   
                     IneurAL (tuned)        0.654360  0.617064     0.685653   
                     Margin                 0.499737  0.346076     0.652471   
                     Neural-CBP             0.696927  0.673456     0.721696   
                     Neuronal (official)    0.683967  0.649076     0.694207   
                     Neuronal (tuned)       0.669588  0.617490     0.683958   
      adult          Cesa                   0.473823  0.346161     0.473823   
                     IneurAL (official)     0.773037  0.707472     0.580468   
                     IneurAL (tuned)        0.636027  0.527199     0.636004   
                     Margin                 0.613309  0.497801     0.633888   
                     Neural-CBP             0.662269  0.575130     0.791835   
                     Neuronal (official)    0.768537  0.675892     0.768254   
                     Neuronal (tuned)       0.788181  0.716209     0.782517   

                                            f1_100 accuracy_250    f1_250  \
                                              mean         mean      mean   
Model Dataset        Approach                                               
MLP   MNISTbinary    Cesa                      NaN          NaN       NaN   
                     IneurAL (official

In [7]:
df.to_csv('experiment.csv', index=False)

In [23]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import plotly.express as px
# Assuming df is your DataFrame and it has columns 'Dataset', 'Value', 'Model'

w = 0.1
def remove_substrings(s, substrings):
    for substring in substrings:
        s = s.replace(substring, '')
    return s


# approaches = result['merge'].unique()
approaches = np.array(['LeNetNeural-CBP', 'LeNetNeuronal (official)', 'LeNetNeuronal (tuned)',
                        'MLPNeural-CBP', 'MLPNeuronal (official)','MLPNeuronal (tuned)', 'MLPIneurAL (official)', 
                        'MLPIneurAL (tuned)',  'MLPCesa', 'MLPMargin' ], dtype=object) 
approaches_names = [ remove_substrings(model, ['LeNet', 'MLP']) for model in approaches ]
default_colors = px.colors.qualitative.Plotly
model_colors = {model_n:col for model_n,col in zip(approaches_names,default_colors) }


# Create an empty figure
fig = go.Figure()


views = []
for appr, appr_name in zip(approaches,approaches_names):

    subset = result[result['merge'] == appr]
    # dec = False
    # if appr_name not in views:
    #     dec = True
    #     views.append(appr_name)
    

    if 'CBP' in appr_name:
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color=model_colors[appr_name],marker_pattern_shape='x',marker_pattern_fgcolor='black', 
            # showlegend = dec,
            # width=w
            ))
    else: 
        fig.add_trace(go.Bar(
            x=subset['merge2'],
            y=subset['Mean'],
            name=appr_name,
            error_y=dict(type='data', array=subset['StdError'], visible=True,thickness=1 ),
            marker_color=model_colors[appr_name], 
            # showlegend = dec,
            # width=w
            ))


# Add vertical black lines between data groups

datasets = result['merge2'].unique()
print(datasets)
for i, dataset in enumerate(datasets[:-1]):
    if i == 2:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black")
    else:
        fig.add_vline(x=i + 0.5, line_width=2, line_color="black",  line_dash="dot")


siz = 13

# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.31,
#     y0=1.01,
#     x1=0.99,
#     y1=1.01,
#     line_width=2,
#     label=dict(text="MLP", font=dict(size=siz+2)),
# )


# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.31,
#     y0=1.0125,
#     x1=0.31,
#     y1=0.95,
#     line_width=2,
# )


# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.99,
#     y0=1.0125,
#     x1=0.99,
#     y1=0.95,
#     line_width=2,
# )



# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.01,
#     y0=1.01,
#     x1=0.29,
#     y1=1.01,
#     line_width=2,
#     label=dict(text="LeNet", font=dict(size=siz+2)),
# )


# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.01,
#     y0=1.0125,
#     x1=0.01,
#     y1=0.95,
#     line_width=2,
# )

# fig.add_shape(
#     type="line", xref="paper", yref="paper",
#     x0=0.29,
#     y0=1.0125,
#     x1=0.29,
#     y1=0.95,
#     line_width=2,
# )

# fig.update_traces(width=0.05)

# fig.update_traces(
#     marker_pattern_shape='/',  # This sets a grid-like pattern. Use '+' for a cross pattern.
#     marker_pattern_fgcolor='black',  # Foreground color of the pattern
#     marker_pattern_bgcolor='rgba(0,0,0,0)'  # Transparent background color for the pattern
# )

fig.update_layout(
    width=1000,
    height=220,
    plot_bgcolor='white',  # Sets the plot background color
    paper_bgcolor='white',  # Sets the overall figure background color
    barmode='group',
    margin=dict(l=20, r=1, t=25, b=20),  # Small margins
    xaxis=dict(
        tickmode='array',
        tickvals=result['merge2'].unique(),
        ticktext = [ remove_substrings(model, ['LeNet', 'MLP']) for model in result['merge2'].unique() ],
        tickfont=dict(size=siz-2)  # Increase X-axis tick font size
    ),
    yaxis=dict(
        gridcolor='lightgrey',
        title="Final regret ± standard dev.",
        title_standoff=10,
        title_font=dict(size=siz),
        tickfont=dict(size=siz)  # Increase Y-axis tick font size
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.3,  # Adjust this value to position the legend
        xanchor="center",
        x=0.5,
        font=dict(size=siz)  # Increase legend font size
    ),
)

fig.show()
# fig.write_image("./results_MLP.pdf" )

['MLPMNIST binary' 'MLPMagicTelescope' 'MLPadult']
